In [24]:
pip install optuna-integration[lightgbm]

Note: you may need to restart the kernel to use updated packages.


In [25]:
# Required: pip install optuna lightgbm -- in Kaggle add as notebook dependency if missing
import re, math, sys
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from lightgbm import LGBMRegressor
import optuna
from optuna.integration import LightGBMPruningCallback
import warnings
warnings.filterwarnings("ignore")

# ---------- Utilities ----------
def detect_pid_col(df):
    for c in df.columns:
        if c.lower() in ('pid','id','sample_id','sampleid'):
            return c
    for c in df.columns:
        if 'pid' in c.lower() or 'id'==c.lower():
            return c
    return None

def detect_sensor_cols(train_cols):
    # patterns to detect common sensor names — returns actual column names in canonical order preference
    patterns = {
        'pH': [r'\bph\b'],
        'Moisture': [r'moist', r'\bsmc\b', r'soil[_\s]?moist'],
        'Temperature': [r'\btemp\b', r'soil[_\s]?temp', r'air[_\s]?temp'],
        'EC': [r'\bec\b', r'electrical[_\s]?conduct', r'electrical[_\s]?conductivity', r'ec_?']
    }
    found = {}
    for k, pats in patterns.items():
        for pat in pats:
            rx = re.compile(pat, flags=re.IGNORECASE)
            for c in train_cols:
                if rx.search(c) and k not in found:
                    found[k] = c
                    break
            if k in found:
                break
    # canonical order we want in features (we will also add env features if present)
    order = ['pH','Moisture','Temperature','EC']
    return [found[k] for k in order if k in found]

def detect_env_features(train_cols):
    # add extra environmental features that help explain nutrients
    candidates = ['lon','lat','alb','bio1','bio12','bio15','bio7','elevation','slope']
    found = [c for c in candidates if c in train_cols]
    # also try fuzzy matches
    for cand in candidates:
        if cand not in found:
            for c in train_cols:
                if cand in c.lower() and c not in found:
                    found.append(c)
    return found

def winsorize_series(s, lower_pct=1, upper_pct=99):
    lo = np.nanpercentile(s, lower_pct)
    hi = np.nanpercentile(s, upper_pct)
    return np.clip(s, lo, hi)

# ---------- Main improved pipeline ----------
def improved_train_pipeline(path_prefix='/kaggle/input/soil-nutrient-gap-prediction-for-sustainable-maize/Soil Nutrient',
                            n_trials_per_nutrient=30,
                            cv_folds=5,
                            random_state=42):
    # 1) load
    train_feats = pd.read_csv(f'{path_prefix}/Train.csv')
    gap_train   = pd.read_csv(f'{path_prefix}/Gap_Train.csv')
    test_feats  = pd.read_csv(f'{path_prefix}/Test.csv')
    sample_sub  = pd.read_csv(f'{path_prefix}/SampleSubmission.csv')

    print("Loaded: Train", train_feats.shape, "Gap_Train", gap_train.shape, "Test", test_feats.shape)

    # 2) find PID and pivot gap (compute Gap if needed)
    pid_gap_col = detect_pid_col(gap_train)
    pid_train_col = detect_pid_col(train_feats)
    pid_test_col = detect_pid_col(test_feats)

    if pid_gap_col is None or pid_train_col is None:
        raise RuntimeError("Can't detect PID columns in gap/train files")

    # compute Gap if not present
    gap_col = next((c for c in gap_train.columns if c.lower()=='gap'), None)
    if gap_col is None:
        req = next((c for c in gap_train.columns if 'required' in c.lower()), None)
        av  = next((c for c in gap_train.columns if 'available' in c.lower()), None)
        if req and av:
            gap_train['Gap'] = pd.to_numeric(gap_train[req], errors='coerce') - pd.to_numeric(gap_train[av], errors='coerce')
            gap_col = 'Gap'
        else:
            raise RuntimeError("No Gap and cannot compute Gap (no Required/Available)")

    nutr_col = next((c for c in gap_train.columns if 'nutr' in c.lower() or 'element' in c.lower()), None)
    if nutr_col is None:
        # fallback: find column that contains values like 'N','P','K' etc.
        for c in gap_train.columns:
            sample_vals = gap_train[c].astype(str).str.strip().str.upper().unique()[:50]
            if any(tok in ['N','P','K','CA','MG','S','FE','MN','ZN','CU','B'] for tok in sample_vals):
                nutr_col = c
                break
    if nutr_col is None:
        raise RuntimeError("Cannot detect Nutrient column in Gap_Train.csv")

    gap_train[pid_gap_col] = gap_train[pid_gap_col].astype(str).str.strip()
    gap_train[nutr_col] = gap_train[nutr_col].astype(str).str.strip().str.upper()

    pivot_gap = gap_train[[pid_gap_col, nutr_col, gap_col]].pivot_table(index=pid_gap_col, columns=nutr_col, values=gap_col, aggfunc='first').reset_index()
    pivot_gap.columns.name = None
    print("Pivoted nutrient columns found:", pivot_gap.columns.tolist()[:40])

    # 3) merge train features with pivoted gap to align samples
    train_feats[pid_train_col] = train_feats[pid_train_col].astype(str).str.strip()
    merged = pd.merge(train_feats, pivot_gap, left_on=pid_train_col, right_on=pid_gap_col, how='left')
    print("Merged shape:", merged.shape)

    # 4) detect features (sensor + env)
    sensor_cols = detect_sensor_cols(train_feats.columns.tolist())
    env_cols = detect_env_features(train_feats.columns.tolist())
    selected_features = sensor_cols + env_cols
    if len(selected_features) == 0:
        raise RuntimeError("No 7-in-1 sensor or env features detected in Train.csv. Columns: " + ", ".join(train_feats.columns))
    print("Using features (sensor + env):", selected_features)

    # 5) basic feature engineering: interactions and squares
    FE = merged[selected_features].copy()
    # create pairwise interactions between sensor features (only sensors)
    for i in range(len(sensor_cols)):
        for j in range(i+1, len(sensor_cols)):
            a = sensor_cols[i]; b = sensor_cols[j]
            if a in FE.columns and b in FE.columns:
                FE[f'{a}_x_{b}'] = FE[a] * FE[b]
    # squares for sensors
    for a in sensor_cols:
        if a in FE.columns:
            FE[f'{a}_sq'] = FE[a] * FE[a]
    # include env cols as-is
    feature_cols = FE.columns.tolist()

    # 6) X imputation
    imputer = SimpleImputer(strategy='median')
    X = pd.DataFrame(imputer.fit_transform(FE), columns=feature_cols)

    # 7) determine target nutrients from pivot_gap (exclude PID)
    pid_col_name = pivot_gap.columns[0]
    target_cols = [c for c in pivot_gap.columns if c != pid_col_name]
    if len(target_cols) == 0:
        raise RuntimeError("No nutrient targets detected after pivot.")
    print("Will attempt to model these nutrients:", target_cols)

    # 8) per-nutrient training with Optuna tuning
    random_seed = random_state
    results = []
    trained_models = {}
    pivot_indexed = pivot_gap.set_index(pid_col_name)

    for nutrient in target_cols:
        print("\n--- Processing nutrient:", nutrient, '---')
        # align y to merged order
        merged_pids = merged[pid_train_col].astype(str).values
        if nutrient not in pivot_indexed.columns:
            print(f"  skipping {nutrient}: not present in pivot index")
            continue
        y_series = pivot_indexed.reindex(merged_pids)[nutrient].reset_index(drop=True)
        y_series = pd.to_numeric(y_series, errors='coerce')

        # remove rows without y for CV training decision, but still keep merged rows order for final mapping
        mask_nonmissing = ~y_series.isna()
        n_samples = int(mask_nonmissing.sum())
        print(f"  samples with target: {n_samples}")
        if n_samples < 50:
            print(f"  skipping {nutrient}: not enough samples (<50) for reliable tuning")
            continue

        # winsorize to reduce extreme outlier influence (1% - 99%)
        y_w = winsorize_series(y_series.values, 1, 99)
        # detect skew, if skew high, use log1p transform
        skewness = pd.Series(y_w).dropna().skew()
        use_log = False
        if abs(skewness) > 1.0:
            use_log = True
            y_proc = np.log1p(np.maximum(y_w, 0))  # avoid negative inside log; if negative values appear, shift min to zero + offset
            # If negative values exist, we shift:
            if np.any(y_w < 0):
                shift = abs(np.nanmin(y_w)) + 1.0
                y_proc = np.log1p(y_w + shift)
            else:
                shift = 0.0
        else:
            y_proc = y_w.copy()
            shift = 0.0

        # Build training arrays for rows where y exists
        X_sub = X.loc[mask_nonmissing].reset_index(drop=True)
        y_sub = pd.Series(y_proc[mask_nonmissing]).reset_index(drop=True)

        # Optuna objective
        def objective(trial):
            param = {
                'objective': 'regression',
                'metric': 'rmse',
                'verbosity': -1,
                'boosting_type': 'gbdt',
                'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
                'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
                'num_leaves': trial.suggest_int('num_leaves', 8, 256),
                'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
                'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.2),
                'n_estimators': 20000,
                'random_state': random_seed
            }
            # small CV for objective (3-fold to speed)
            kf_small = KFold(n_splits=3, shuffle=True, random_state=random_seed)
            rmses = []
            for tr_idx, val_idx in kf_small.split(X_sub):
                X_tr, X_val = X_sub.iloc[tr_idx], X_sub.iloc[val_idx]
                y_tr, y_val = y_sub.iloc[tr_idx].values, y_sub.iloc[val_idx].values
                model = LGBMRegressor(**param)
                # use callbacks for early stopping
                model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                          callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(period=0)])
                y_pred = model.predict(X_val)
                rmses.append(mean_squared_error(y_val, y_pred, squared=False))
            return float(np.mean(rmses))

        # run optuna
        study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=random_seed),
                                    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5))
        n_trials = max(20, n_trials_per_nutrient)  # set a reasonable number; tuneable
        study.optimize(objective, n_trials=n_trials, show_progress_bar=False)
        best_params = study.best_params
        print("  Optuna best params (partial):", {k: best_params[k] for k in list(best_params)[:6]})

        # train final model on full available y_sub with best params
        final_param = {
            'objective': 'regression',
            'metric': 'rmse',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'random_state': random_seed,
            'n_estimators': 20000,
            'learning_rate': best_params.get('learning_rate', 0.05),
            'num_leaves': best_params.get('num_leaves', 31),
            'feature_fraction': best_params.get('feature_fraction', 0.8),
            'bagging_fraction': best_params.get('bagging_fraction', 0.8),
            'bagging_freq': best_params.get('bagging_freq', 1),
            'min_child_samples': best_params.get('min_child_samples', 20),
            'lambda_l1': best_params.get('lambda_l1', 0.0),
            'lambda_l2': best_params.get('lambda_l2', 0.0)
        }
        final_model = LGBMRegressor(**final_param)
        final_model.fit(X_sub, y_sub, eval_set=[(X_sub, y_sub)],
                        callbacks=[lgb.early_stopping(stopping_rounds=200), lgb.log_evaluation(period=0)])

        # Cross-validated RMSE computed in original units:
        kf_full = KFold(n_splits=cv_folds, shuffle=True, random_state=random_seed)
        cv_rmses = []
        for tr_idx, val_idx in kf_full.split(X_sub):
            X_tr, X_val = X_sub.iloc[tr_idx], X_sub.iloc[val_idx]
            y_tr, y_val = y_sub.iloc[tr_idx].values, y_sub.iloc[val_idx].values
            m = LGBMRegressor(**final_param)
            m.fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                  callbacks=[lgb.early_stopping(stopping_rounds=200), lgb.log_evaluation(period=0)])
            y_pred_scaled = m.predict(X_val)
            # inverse transform to original units
            if use_log:
                # inverse log1p (and undo shift if used)
                if shift != 0.0:
                    y_pred_orig = np.expm1(y_pred_scaled) - shift
                    y_true_orig = np.expm1(y_val) - shift
                else:
                    y_pred_orig = np.expm1(y_pred_scaled)
                    y_true_orig = np.expm1(y_val)
            else:
                y_pred_orig = y_pred_scaled
                y_true_orig = y_val
            # if we winsorized earlier, we compare on winsorized scale (consistent)
            cv_rmses.append(mean_squared_error(y_true_orig, y_pred_orig, squared=False))
        mean_rmse = float(np.mean(cv_rmses))
        std_rmse = float(np.std(cv_rmses))

        results.append({'nutrient': nutrient, 'cv_rmse_mean': mean_rmse, 'cv_rmse_std': std_rmse, 'n_samples': n_samples, 'use_log': use_log})
        trained_models[nutrient] = {'model': final_model, 'use_log': use_log, 'shift': shift, 'feature_cols': feature_cols}
        print(f"  -> Completed {nutrient}: CV RMSE = {mean_rmse:.4f} ± {std_rmse:.4f}")

    # Summary
    res_df = pd.DataFrame(results).sort_values('cv_rmse_mean')
    print("\n=== Final CV RMSE Summary (after tuning & transforms) ===")
    for _, r in res_df.iterrows():
        print(f" - {r['nutrient']}: CV RMSE = {r['cv_rmse_mean']:.4f} ± {r['cv_rmse_std']:.4f} (n={r['n_samples']}) use_log={r['use_log']}")

    return {
        'trained_models': trained_models,
        'results': res_df,
        'imputer': imputer,
        'feature_cols': feature_cols,
        'merged': merged,
        'pivot_gap': pivot_gap
    }

# Run it:
out = improved_train_pipeline(n_trials_per_nutrient=30, cv_folds=5, random_state=42)


[I 2025-10-12 06:26:42,837] A new study created in memory with name: no-name-ec28c535-2233-468f-bbd2-d3f499b802c6


Loaded: Train (7744, 44) Gap_Train (85184, 5) Test (2418, 33)
Pivoted nutrient columns found: ['PID', 'B', 'CA', 'CU', 'FE', 'K', 'MG', 'MN', 'N', 'P', 'S', 'ZN']
Merged shape: (7744, 55)
Using features (sensor + env): ['pH', 'cec20', 'lon', 'lat', 'alb', 'bio1', 'bio12', 'bio15', 'bio7', 'slope']
Will attempt to model these nutrients: ['B', 'CA', 'CU', 'FE', 'K', 'MG', 'MN', 'N', 'P', 'S', 'ZN']

--- Processing nutrient: B ---
  samples with target: 7744
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[50]	valid_0's rmse: 0.127278
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[36]	valid_0's rmse: 0.113164
Training until validation scores don't improve for 100 rounds


[I 2025-10-12 06:26:45,553] Trial 0 finished with value: 0.11954588408126388 and parameters: {'lambda_l1': 2.348881295853308e-05, 'lambda_l2': 3.6010467344475403, 'num_leaves': 190, 'feature_fraction': 0.759195090518222, 'bagging_fraction': 0.4936111842654619, 'bagging_freq': 2, 'min_child_samples': 10, 'learning_rate': 0.09842315738502598}. Best is trial 0 with value: 0.11954588408126388.


Early stopping, best iteration is:
[40]	valid_0's rmse: 0.118196
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3078]	valid_0's rmse: 0.127315
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4696]	valid_0's rmse: 0.1143
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4674]	valid_0's rmse: 0.118706


[I 2025-10-12 06:26:58,786] Trial 1 finished with value: 0.12010690225341676 and parameters: {'lambda_l1': 0.002570603566117598, 'lambda_l2': 0.023585940584142682, 'num_leaves': 13, 'feature_fraction': 0.9819459112971965, 'bagging_fraction': 0.899465584480253, 'bagging_freq': 2, 'min_child_samples': 22, 'learning_rate': 0.002642526057549917}. Best is trial 0 with value: 0.11954588408126388.


...

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1170]	valid_0's rmse: 0.293463
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1430]	valid_0's rmse: 0.267028
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1062]	valid_0's rmse: 0.25768


[I 2025-10-12 08:49:03,129] Trial 28 finished with value: 0.2727234685435911 and parameters: {'lambda_l1': 1.1083316872538955, 'lambda_l2': 1.0305905796854952e-07, 'num_leaves': 238, 'feature_fraction': 0.7200362957650038, 'bagging_fraction': 0.47993692893740536, 'bagging_freq': 5, 'min_child_samples': 5, 'learning_rate': 0.002381576888315121}. Best is trial 25 with value: 0.2718986206789307.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[79]	valid_0's rmse: 0.293554
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[76]	valid_0's rmse: 0.268447
Training until validation scores don't improve for 100 rounds


[I 2025-10-12 08:49:05,938] Trial 29 finished with value: 0.2736880028514796 and parameters: {'lambda_l1': 0.07855056760033415, 'lambda_l2': 8.004787547021176e-05, 'num_leaves': 203, 'feature_fraction': 0.7776097797023942, 'bagging_fraction': 0.4822880447489878, 'bagging_freq': 3, 'min_child_samples': 13, 'learning_rate': 0.03359625216923769}. Best is trial 25 with value: 0.2718986206789307.


Early stopping, best iteration is:
[117]	valid_0's rmse: 0.259063
  Optuna best params (partial): {'lambda_l1': 0.23244637014573577, 'lambda_l2': 2.145503697180748e-06, 'num_leaves': 256, 'feature_fraction': 0.811436803699906, 'bagging_fraction': 0.473557099189081, 'bagging_freq': 5}
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1635]	valid_0's rmse: 0.28992
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1541]	valid_0's rmse: 0.279375
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[2100]	valid_0's rmse: 0.269085
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1350]	valid_0's rmse: 0.265109
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1760]	valid_0's rmse: 0.253534
  -> Completed ZN: CV RMSE = 3.1381 ± 0.0531

=== Final CV RMSE Summary

In [27]:
import numpy as np
import pandas as pd
import joblib

# Load saved model bundle (from training phase)
bundle = joblib.load('trained_sensor_model.pkl')
model = bundle['model']
imputer = bundle['imputer']
feature_cols = bundle['feature_cols']
target_cols = bundle['target_cols']

# --- Example manual samples (realistic 7-in-1 soil sensor readings) ---
manual_examples = [
    {'pH': 7.2, 'Moisture': 0.25, 'Temperature': 22.5, 'EC': 0.8},   # example 1
    {'pH': 6.5, 'Moisture': 0.40, 'Temperature': 28.0, 'EC': 1.5},   # example 2
]

# --- Helper: map canonical names to dataset feature columns ---
def map_example_to_features(example, feature_cols):
    row = {}
    for col in feature_cols:
        lc = col.lower()
        if 'ph' in lc:
            row[col] = example.get('pH', np.nan)
        elif 'moist' in lc or 'smc' in lc:
            row[col] = example.get('Moisture', np.nan)
        elif 'temp' in lc:
            row[col] = example.get('Temperature', np.nan)
        elif 'ec' in lc or 'electrical' in lc or 'cec' in lc:
            row[col] = example.get('EC', np.nan)
        else:
            # fill NaN for unrelated features
            row[col] = np.nan
    return row

# --- Build DataFrame in the same order as during training ---
manual_rows = [map_example_to_features(ex, feature_cols) for ex in manual_examples]
X_manual_raw = pd.DataFrame(manual_rows, columns=feature_cols)

print("Manual input (aligned with training features):")
print(X_manual_raw.head())

# --- Impute missing values using same imputer ---
X_manual = pd.DataFrame(imputer.transform(X_manual_raw), columns=feature_cols)

# --- Predict nutrient gaps ---
preds_manual = model.predict(X_manual)
preds_df = pd.DataFrame(preds_manual, columns=target_cols)
preds_df.insert(0, 'sample_idx', range(1, len(preds_df)+1))

print("\nPredicted Nutrient Gaps (same units as dataset):")
print(preds_df)

# --- Pretty-print for readability ---
for i, row in preds_df.iterrows():
    print(f"\nSample {int(row['sample_idx'])}:")
    for nutr in target_cols:
        print(f"  {nutr}: {row[nutr]:.3f}")


Manual input (aligned with training features):
    pH  cec20
0  7.2    0.8
1  6.5    1.5

Predicted Nutrient Gaps (same units as dataset):
   sample_idx         B           CA        CU          FE            K  \
0           1 -2.071711 -8335.698266 -2.173094 -230.487697 -1072.279465   
1           2 -1.696197 -3652.677618 -2.074561 -264.445538  -890.189003   

            MG          MN            N         P          S         ZN  
0   957.501936 -491.026269 -5683.233750 -7.537174 -25.541448 -12.545983  
1  1205.461697 -451.537987 -6121.877086  6.083292 -25.558297 -12.301425  

Sample 1:
  B: -2.072
  CA: -8335.698
  CU: -2.173
  FE: -230.488
  K: -1072.279
  MG: 957.502
  MN: -491.026
  N: -5683.234
  P: -7.537
  S: -25.541
  ZN: -12.546

Sample 2:
  B: -1.696
  CA: -3652.678
  CU: -2.075
  FE: -264.446
  K: -890.189
  MG: 1205.462
  MN: -451.538
  N: -6121.877
  P: 6.083
  S: -25.558
  ZN: -12.301
